## Import Libraries


In [1]:
import torch
import unsloth
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
from unsloth import FastVisionModel
import os
import pandas as pd
import numpy as np



KeyboardInterrupt: 

# Load the Dataset

In [2]:
json_path=r"./vlm_dataset.jsonl"
from sklearn.model_selection import train_test_split
import json 
from tqdm import tqdm
import random
#for Train, Eval and Test split
# dataset_train=load_dataset("json", data_files=json_path,split="train[20%:]")
# print(dataset_train[1:5])
# print(len(dataset_train))



# dataset_eval=load_dataset("json", data_files=json_path,split="train[:10%]")
# print(dataset_eval[1:5])
# print(len(dataset_eval))

# dataset_test=load_dataset("json", data_files=json_path,split="train[10%:20%]")
# print(dataset_test[1:5])
# print(len(dataset_test))
# dataset_train, dataset_eval=load_dataset("json", data_files=json_path,split="train").train_test_split(test_size=0.2).values()
# dataset_train[1:5]
# print(len(dataset))
# train, test = train_test_split(dataset, test_size=0.2)


TRAIN_RATIO = 0.8
EVAL_RATIO = 0.1
TEST_RATIO = 0.1
with open(json_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in tqdm(f, desc="Loading dataset")]
random.seed(42)
random.shuffle(data)

n_total = len(data)
n_train = int(n_total * TRAIN_RATIO)
n_eval = int(n_total * EVAL_RATIO)

train_data = data[:n_train]
eval_data = data[n_train:n_train + n_eval]
test_data = data[n_train + n_eval:]
with open("test_data.jsonl", "w", encoding="utf-8") as f:
    for item in test_data:
        f.write(json.dumps(item) + "\n")
with open("eval_data.jsonl", "w", encoding="utf-8") as f:
    for item in eval_data:
        f.write(json.dumps(item) + "\n")
with open("train_data.jsonl", "w", encoding="utf-8") as f:
    for item in train_data:
        f.write(json.dumps(item) + "\n")
print(f"Train: {len(train_data)}, Eval: {len(eval_data)}, Test: {len(test_data)}")
print("Sample train data:\n", train_data[0:5])


Loading dataset: 81725it [00:01, 44968.54it/s]


Train: 65380, Eval: 8172, Test: 8173
Sample train data:
 [{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': "You are highly advance Vision Language Model, specializing in analyzing, describing and interpreting medical images .You are given a medical image and a question related to it. Your task is to analyze, process and choose the most appropriate answer from the options provided.\n Question: In arterial tortuosity syndrome, where does the distal aorta tend to extend as observed in the figure? \n Options:\n A. To the right pulmonary artery\nB. To the left hemithorax\nC. Toward the heart's apex\nD. Upward towards the neck\nE. Into the right atrium\nF. Toward the diaphragm"}, {'type': 'image', 'image': 'C:/Users/Hey-BUDD/Desktop/Deep Learining Project/Datasets/RadFig-VQA/imgs/imgs/PMC4168646/JCIS-4-44-g012.jpg'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'B. To the left hemithorax'}]}]}, {'messages': [{'role': 'user', 'content': [{'type': 'text', 'te

In [ ]:
train_data[0:5]



[{'messages': [{'role': 'user',
    'content': [{'type': 'text',
      'text': "You are highly advance Vision Language Model, specializing in analyzing, describing and interpreting medical images .You are given a medical image and a question related to it. Your task is to analyze, process and choose the most appropriate answer from the options provided.\n Question: In arterial tortuosity syndrome, where does the distal aorta tend to extend as observed in the figure? \n Options:\n A. To the right pulmonary artery\nB. To the left hemithorax\nC. Toward the heart's apex\nD. Upward towards the neck\nE. Into the right atrium\nF. Toward the diaphragm"},
     {'type': 'image',
      'image': 'C:/Users/Hey-BUDD/Desktop/Deep Learining Project/Datasets/RadFig-VQA/imgs/imgs/PMC4168646/JCIS-4-44-g012.jpg'}]},
   {'role': 'assistant',
    'content': [{'type': 'text', 'text': 'B. To the left hemithorax'}]}]},
 {'messages': [{'role': 'user',
    'content': [{'type': 'text',
      'text': 'You are high

In [ ]:
import json

with open(json_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f, start=1):
        data = json.loads(line)
        user_contents = data["messages"][0]["content"]
        images = [x for x in user_contents if x["type"] == "image"]
        if not images or not images[0]["image"]:
            print(f"⚠️ Missing image at line {i}")
import json, re

with open(json_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f, 1):
        data = json.loads(line)
        text = " ".join([x["text"] for x in data["messages"][0]["content"] if x["type"] == "text"])
        if re.search(r"<\|?image", text):
            print(f"⚠️ Found image placeholder in text at line {i}")

import json

with open(json_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f, 1):
        data = json.loads(line)
        imgs = [c for m in data["messages"] for c in m["content"] if c["type"] == "image"]
        if len(imgs) > 1:
            print(f"⚠️ Multiple images at line {i}: {[x['image'] for x in imgs]}")


In [ ]:
from unsloth import FastVisionModel 
model,tokenizer=FastVisionModel.from_pretrained("unsloth/Qwen2.5-VL-3B-Instruct-bnb-4bit",
    max_seq_length=4096,
    dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
    device_map="auto",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth")



Device does not support bfloat16. Will change to float16.
c:\Users\Hey-BUDD\AppData\Local\Programs\Python\Python312\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:341: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.10.1: Fast Qwen2_5_Vl patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 2060 SUPER. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True, #
    finetune_attention_modules = True, 
    finetune_mlp_modules       = True,

    r = 16,            
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [ ]:
output="./outputs/Qwen2.5-VL-3B-Instruct-bnb-4bit-finetuned-r10"
from trl import SFTTrainer, SFTConfig

args = SFTConfig(
        # per_device_train_batch_size = 8,
        # gradient_accumulation_steps = 4,
        warmup_steps = 100,
        max_steps = 4000,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-5,
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output,
        report_to = "none",     # For Weights and Biases
        # eval_strategy = "steps",
        metric_for_best_model = "loss",
        # load_best_model_at_end = True,
        greater_is_better = False,
        log_level = "info",
        # resume_from_checkpoint = True,   #"C://Users//Hey-BUDD//Desktop//Deep Learining Project//Datasets//RadFig-VQA//outputs//Qwen2.5-VL-3B-Instruct-bnb-4bit-finetuned-r10//checkpoint-300", # or path to checkpoint
        
        save_strategy ="steps",
        save_steps = 100,
        max_grad_norm = 1.0,    
        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 4096,      
    )

PyTorch: setting up devices
Both warmup_ratio and warmup_steps given, warmup_steps will override any effect of warmup_ratio during training


In [ ]:
from unsloth.trainer import UnslothVisionDataCollator

FastVisionModel.for_training(model)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    
    train_dataset = train_data,
    eval_dataset = eval_data,     
    args = args
    
) 

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


Unsloth: Model does not have a default image size - using 512


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 2060 SUPER. Max memory = 8.0 GB.
11.533 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint=True)


Loading model from ./outputs/Qwen2.5-VL-3B-Instruct-bnb-4bit-finetuned-r10\checkpoint-3000.
skipped Embedding(151936, 2048, padding_idx=151654): 296.75M params
skipped: 296.75M params
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 65,380 | Num Epochs = 1 | Total steps = 4,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 41,084,928 of 3,795,707,904 (1.08% trained)
	save_steps: 100 (from args) != 50 (from trainer_state.json)
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 3000
  Will skip the first 0 epochs then the first 6000 batches in the first epoch.


Step,Training Loss


RuntimeError: setStorage: sizes [12984320], strides [1], storage offset 0, and itemsize 2 requiring a storage size of 25968640 are out of bounds for storage of size 24363008

In [1]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'torch' is not defined

In [ ]:
FastVisionModel.for_inference(model) # Enable for inference!

image = "C:/Users/Hey-BUDD/Desktop/Deep Learining Project/Datasets/RadFig-VQA/imgs/imgs/PMC6604293/12880_2019_349_Fig1_HTML.jpg"
question = "What imaging technique is used to display the lesions shown in the figure?"

instruction = "You are an expert radiographer. Describe accurately what you see in this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction+"\n"+question}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Abdominal ultrasound (US) was used. The figure focuses on the lesion in the upper left part of the right adrenal gland, measured with a diameter of approximately 13mm in diameter. The right adrenal gland also contains a malignant lesion, approximately 10×9 mm2. These findings were confirmed during surgery. Additionally, the lower left adrenals contain several small benign and malignant masses, as well as cystic structures in both adrenal glands. There was no evidence of metastasis in lymph nodes. The lesions were located within the adrenal glands according to imaging and post-operative histological examination. The largest benign nodule in


In [ ]:
model.save_pretrained("Qwen_25_VL_Finetuned_RadFigv2")  # Local saving
tokenizer.save_pretrained("Qwen_25_VL_Finetuned_RadFigv2")

loading configuration file config.json from cache at C:\Users\Hey-BUDD\.cache\huggingface\hub\models--unsloth--qwen2.5-vl-3b-instruct-bnb-4bit\snapshots\f9adca15cefb612e7f03a1c8a5fddb920b1786f3\config.json
Model config Qwen2_5_VLConfig {
  "architectures": [
    "Qwen2_5_VLForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "image_token_id": 151655,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 128000,
  "max_window_layers": 70,
  "model_type": "qwen2_5_vl",
  "num_attention_heads": 16,
  "num_hidden_layers": 36,
  "num_key_value_heads": 2,
  "pad_token_id": 151654,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": fals

[]